Competition link : https://www.kaggle.com/competitions/home-data-for-ml-course/leaderboard?

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

## Model evaluators
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
# from sklearn.metrics import plot_roc_curve # note: this was changed in Scikit-Learn 1.2+ to be "RocCurveDisplay" (see below)
from sklearn.metrics import RocCurveDisplay # new in Scikit-Learn 1.2+


In [5]:
train = pd.read_csv(r"C:\Users\navad\Desktop\ML\2.House_price_Pred\train.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
train.shape

(1460, 81)

In [7]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
for label,content in train.items():
    if pd.api.types.is_string_dtype(content):
        train[label] = content.astype("category").cat.as_ordered()

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1460 non-null   int64   
 1   MSSubClass     1460 non-null   int64   
 2   MSZoning       1460 non-null   category
 3   LotFrontage    1201 non-null   float64 
 4   LotArea        1460 non-null   int64   
 5   Street         1460 non-null   category
 6   Alley          91 non-null     object  
 7   LotShape       1460 non-null   category
 8   LandContour    1460 non-null   category
 9   Utilities      1460 non-null   category
 10  LotConfig      1460 non-null   category
 11  LandSlope      1460 non-null   category
 12  Neighborhood   1460 non-null   category
 13  Condition1     1460 non-null   category
 14  Condition2     1460 non-null   category
 15  BldgType       1460 non-null   category
 16  HouseStyle     1460 non-null   category
 17  OverallQual    1460 non-null   in

In [11]:
for label,content in train.items():
    if pd.api.types.is_object_dtype(content):
        train[label] = content.astype("category").cat.as_ordered()

In [12]:
train.MSZoning.cat.codes

0       3
1       3
2       3
3       3
4       3
       ..
1455    3
1456    3
1457    3
1458    3
1459    3
Length: 1460, dtype: int8

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1460 non-null   int64   
 1   MSSubClass     1460 non-null   int64   
 2   MSZoning       1460 non-null   category
 3   LotFrontage    1201 non-null   float64 
 4   LotArea        1460 non-null   int64   
 5   Street         1460 non-null   category
 6   Alley          91 non-null     category
 7   LotShape       1460 non-null   category
 8   LandContour    1460 non-null   category
 9   Utilities      1460 non-null   category
 10  LotConfig      1460 non-null   category
 11  LandSlope      1460 non-null   category
 12  Neighborhood   1460 non-null   category
 13  Condition1     1460 non-null   category
 14  Condition2     1460 non-null   category
 15  BldgType       1460 non-null   category
 16  HouseStyle     1460 non-null   category
 17  OverallQual    1460 non-null   in

In [14]:
for label,content in train.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [15]:
for label,content in train.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

Alley
MasVnrType
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Electrical
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature


In [16]:
pd.Categorical(train["BsmtQual"]).codes

array([2, 2, 2, ..., 3, 3, 3], dtype=int8)

In [17]:
L = list(pd.Categorical(train["BsmtQual"]).codes)
for i in range(len(L)):
    if L[i] == -1:
        print(i)

17
39
90
102
156
182
259
342
362
371
392
520
532
533
553
646
705
736
749
778
868
894
897
984
1000
1011
1035
1045
1048
1049
1090
1179
1216
1218
1232
1321
1412


In [18]:
pd.Categorical(train["PoolQC"]).codes

array([-1, -1, -1, ..., -1, -1, -1], dtype=int8)

Here PoolQC column has missing values and because of this it fills the codes for the missing values as -1. So to have some code value so that no error comes while we fit the data to a model, we need to fill the missing values which is done in the following cell.

In [19]:
for label,content in train.items():
    if not pd.api.types.is_numeric_dtype(content):
        train[label] = content.astype("category").cat.codes
        
        if pd.isnull(content).sum():
            #train[label+"_is_missing"] = pd.isnull(content).astype(int)
            train[label] = pd.Categorical(content).codes + 1

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int8   
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int8   
 6   Alley          1460 non-null   int8   
 7   LotShape       1460 non-null   int8   
 8   LandContour    1460 non-null   int8   
 9   Utilities      1460 non-null   int8   
 10  LotConfig      1460 non-null   int8   
 11  LandSlope      1460 non-null   int8   
 12  Neighborhood   1460 non-null   int8   
 13  Condition1     1460 non-null   int8   
 14  Condition2     1460 non-null   int8   
 15  BldgType       1460 non-null   int8   
 16  HouseStyle     1460 non-null   int8   
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [21]:
pd.Categorical(train["PoolQC"]).codes

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [22]:
pd.Categorical(train["BsmtQual"]).codes

array([3, 3, 3, ..., 4, 4, 4], dtype=int8)

In [23]:
L = list(pd.Categorical(train["BsmtQual"]).codes)
for i in range(len(L)):
    if L[i] == -1:
        print(i)

In [24]:
train.shape

(1460, 81)

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int8   
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int8   
 6   Alley          1460 non-null   int8   
 7   LotShape       1460 non-null   int8   
 8   LandContour    1460 non-null   int8   
 9   Utilities      1460 non-null   int8   
 10  LotConfig      1460 non-null   int8   
 11  LandSlope      1460 non-null   int8   
 12  Neighborhood   1460 non-null   int8   
 13  Condition1     1460 non-null   int8   
 14  Condition2     1460 non-null   int8   
 15  BldgType       1460 non-null   int8   
 16  HouseStyle     1460 non-null   int8   
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [26]:
X = train.drop("SalePrice",axis = 1)
Y = train["SalePrice"]
X.shape, Y.shape

((1460, 80), (1460,))

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid = train_test_split(X,Y,test_size = 0.2)

In [28]:
X_train.shape,Y_train.shape,X_valid.shape,Y_valid.shape

((1168, 80), (1168,), (292, 80), (292,))

In [29]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1168 entries, 1073 to 980
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1168 non-null   int64  
 1   MSSubClass     1168 non-null   int64  
 2   MSZoning       1168 non-null   int8   
 3   LotFrontage    964 non-null    float64
 4   LotArea        1168 non-null   int64  
 5   Street         1168 non-null   int8   
 6   Alley          1168 non-null   int8   
 7   LotShape       1168 non-null   int8   
 8   LandContour    1168 non-null   int8   
 9   Utilities      1168 non-null   int8   
 10  LotConfig      1168 non-null   int8   
 11  LandSlope      1168 non-null   int8   
 12  Neighborhood   1168 non-null   int8   
 13  Condition1     1168 non-null   int8   
 14  Condition2     1168 non-null   int8   
 15  BldgType       1168 non-null   int8   
 16  HouseStyle     1168 non-null   int8   
 17  OverallQual    1168 non-null   int64  
 18  OverallCond

In [30]:
for label,content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

Id
MSSubClass
MSZoning
LotFrontage
LotArea
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
MasVnrArea
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
Heating
HeatingQC
CentralAir
Electrical
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageArea
GarageQual
GarageCond
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
PoolQC
Fence
MiscFeature
MiscVal
MoSold
YrSold
SaleType
SaleCondition


In [31]:
for label,content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

LotFrontage
MasVnrArea
GarageYrBlt


In [32]:
for label,content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # X_train[label+"_is_missing"] = pd.isnull(content).astype(int)
            X_train[label] = content.fillna(content.median())

In [33]:
for label,content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [34]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1168 entries, 1073 to 980
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1168 non-null   int64  
 1   MSSubClass     1168 non-null   int64  
 2   MSZoning       1168 non-null   int8   
 3   LotFrontage    1168 non-null   float64
 4   LotArea        1168 non-null   int64  
 5   Street         1168 non-null   int8   
 6   Alley          1168 non-null   int8   
 7   LotShape       1168 non-null   int8   
 8   LandContour    1168 non-null   int8   
 9   Utilities      1168 non-null   int8   
 10  LotConfig      1168 non-null   int8   
 11  LandSlope      1168 non-null   int8   
 12  Neighborhood   1168 non-null   int8   
 13  Condition1     1168 non-null   int8   
 14  Condition2     1168 non-null   int8   
 15  BldgType       1168 non-null   int8   
 16  HouseStyle     1168 non-null   int8   
 17  OverallQual    1168 non-null   int64  
 18  OverallCond

We should be doing the same transformations to the valid data also and fit the training data and evaluate the model with the valid data

In [35]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292 entries, 881 to 1143
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             292 non-null    int64  
 1   MSSubClass     292 non-null    int64  
 2   MSZoning       292 non-null    int8   
 3   LotFrontage    237 non-null    float64
 4   LotArea        292 non-null    int64  
 5   Street         292 non-null    int8   
 6   Alley          292 non-null    int8   
 7   LotShape       292 non-null    int8   
 8   LandContour    292 non-null    int8   
 9   Utilities      292 non-null    int8   
 10  LotConfig      292 non-null    int8   
 11  LandSlope      292 non-null    int8   
 12  Neighborhood   292 non-null    int8   
 13  Condition1     292 non-null    int8   
 14  Condition2     292 non-null    int8   
 15  BldgType       292 non-null    int8   
 16  HouseStyle     292 non-null    int8   
 17  OverallQual    292 non-null    int64  
 18  OverallCond 

In [36]:
for label,content in X_valid.items():
    if pd.api.types.is_numeric_dtype(content):
        
        if pd.isnull(content).sum():
            #X_valid[label+"_is_missing"] = pd.isnull(content).astype(int)
            X_valid[label] = content.fillna(content.median())

In [37]:
for label,content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [38]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292 entries, 881 to 1143
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             292 non-null    int64  
 1   MSSubClass     292 non-null    int64  
 2   MSZoning       292 non-null    int8   
 3   LotFrontage    292 non-null    float64
 4   LotArea        292 non-null    int64  
 5   Street         292 non-null    int8   
 6   Alley          292 non-null    int8   
 7   LotShape       292 non-null    int8   
 8   LandContour    292 non-null    int8   
 9   Utilities      292 non-null    int8   
 10  LotConfig      292 non-null    int8   
 11  LandSlope      292 non-null    int8   
 12  Neighborhood   292 non-null    int8   
 13  Condition1     292 non-null    int8   
 14  Condition2     292 non-null    int8   
 15  BldgType       292 non-null    int8   
 16  HouseStyle     292 non-null    int8   
 17  OverallQual    292 non-null    int64  
 18  OverallCond 

In [39]:
X_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1073,1074,60,3,75.0,7950,1,0,0,0,0,...,0,0,0,3,0,0,7,2009,8,4
649,650,180,4,21.0,1936,1,0,3,3,0,...,0,0,0,3,0,0,12,2007,8,4
811,812,120,4,69.0,4438,1,0,3,3,0,...,0,0,0,0,0,0,6,2008,3,4
30,31,70,0,50.0,8500,1,2,3,3,0,...,0,0,0,3,0,0,7,2008,8,4
919,920,20,3,87.0,11029,1,0,0,3,0,...,190,0,0,0,0,0,5,2008,8,4


In [40]:
X_valid.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
881,882,50,3,44.0,13758,1,0,0,3,0,...,0,0,0,0,0,0,4,2007,8,4
145,146,160,4,24.0,2522,1,0,3,3,0,...,0,0,0,0,0,0,4,2006,8,4
774,775,20,3,110.0,14226,1,0,3,3,0,...,0,0,0,0,0,0,7,2007,6,5
34,35,120,3,60.0,7313,1,0,3,3,0,...,0,0,0,0,0,0,8,2007,8,4
333,334,120,4,59.0,8198,1,0,3,3,0,...,0,0,0,0,0,0,7,2008,8,4


The training and validation data is all set.Now let's make a function of models and fit the X_train,Y_train data and make an evaluation function and evaluate with X_valid,Y_valid

In [41]:
%%time
model = RandomForestRegressor(n_jobs=-1,random_state=42)
model.fit(X_train,Y_train)

CPU times: total: 5.12 s
Wall time: 1.62 s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [76]:
#evaluate function
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
def rmsle(x,y):
    return np.sqrt(mean_squared_log_error(x,y))
def scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    sc = {"train_rmsle" : rmsle(Y_train,train_preds),
          "val_rmsle"   : rmsle(Y_valid,val_preds),
          "train_r2"    :  r2_score(Y_train, train_preds),
          "valid_r2"    : r2_score(Y_valid,val_preds) }
    return sc
    

In [77]:
scores(model)

{'train_rmsle': 0.01420858430218418,
 'val_rmsle': 0.1259427253137462,
 'train_r2': 0.9993463754653931,
 'valid_r2': 0.9041399359703064}

In [45]:
test = pd.read_csv(r"C:\Users\navad\Desktop\ML\2.House_price_Pred\test.csv")

In [46]:
test.shape

(1459, 80)

In [47]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [48]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [49]:
for label,content in test.items():
    if not pd.api.types.is_numeric_dtype(content):
        test[label] = content.astype("category").cat.codes
        if pd.isnull(content).sum():
            #test[label+"_is_missing"] = pd.isnull(content).astype(int)
            test[label] = pd.Categorical(content).codes + 1

for label,content in test.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #test[label+"_is_missing"] = pd.isnull(content).astype(int)
            test[label] = content.fillna(content.median())

In [50]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   int8   
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   int8   
 6   Alley          1459 non-null   int8   
 7   LotShape       1459 non-null   int8   
 8   LandContour    1459 non-null   int8   
 9   Utilities      1459 non-null   int8   
 10  LotConfig      1459 non-null   int8   
 11  LandSlope      1459 non-null   int8   
 12  Neighborhood   1459 non-null   int8   
 13  Condition1     1459 non-null   int8   
 14  Condition2     1459 non-null   int8   
 15  BldgType       1459 non-null   int8   
 16  HouseStyle     1459 non-null   int8   
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [51]:
test_preds = model.predict(test)

In [52]:
test.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [53]:
test["Id"]

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [54]:
output = pd.DataFrame({"Id" : test["Id"],"SalePrice" : test_preds })

In [55]:
output

,Id,SalePrice
0,1461,127388.08
1,1462,154536.00
2,1463,179369.64
3,1464,176946.00
4,1465,208851.98
...,...,...
1454,2915,87232.54
1455,2916,90438.51
1456,2917,152148.09
1457,2918,115452.00


In [56]:
output.to_csv(r"C:\Users\navad\Desktop\2.House_price_Pred\RFR_model_submission.csv")

OSError: Cannot save file into a non-existent directory: 'C:\Users\navad\Desktop\2.House_price_Pred'

In [ ]:
output.to_csv('submissionRFR.csv')

In [57]:
%%time
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1.0, "sqrt"], # Note: "max_features='auto'" is equivalent to "max_features=1.0", as of Scikit-Learn version 1.1
           "max_samples": [None, 0.5, 0.75, 1.0]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions=rf_grid,
                              n_iter=100,
                              cv=5,
                              verbose=True)

rs_model.fit(X_train, Y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: total: 3min 24s
Wall time: 3min 33s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1.0, 'sqrt'],
                                        'max_samples': [None, 0.5, 0.75, 1.0],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   verbose=True)

In [79]:
# Evaluate the RandomizedSearch model
scores(rs_model)

{'train_rmsle': 0.10464695902164166,
 'val_rmsle': 0.133816272688547,
 'train_r2': 0.9270571315543706,
 'valid_r2': 0.8856236748893407}

In [70]:
%%time
# Most ideal hyperparameters
ideal_model = RandomForestRegressor(n_estimators=200,
                                    min_samples_leaf=1,
                                    min_samples_split=14,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None)
ideal_model.fit(X_train, Y_train)

CPU times: total: 3.92 s
Wall time: 1.2 s


RandomForestRegressor(max_features=0.5, min_samples_split=14, n_estimators=200,
                      n_jobs=-1)

In [71]:
scores(ideal_model)

{'train_rmsle': 0.09216501442696506, 'val_rmsle': 0.14952115036884608}

In [77]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 5.7 MB/s eta 0:00:22
   ---------------------------------------- 1.0/124.9 MB 2.7 MB/s eta 0:00:47
   ---------------------------------------- 1.0/124.9 MB 2.7 MB/s eta 0:00:47
    --------------------------------------- 1.6/124.9 MB 1.7 MB/s eta 0:01:11
    --------------------------------------- 2.1/124.9 MB 1.9 MB/s eta 0:01:04
    --------------------------------------- 2.6/124.9 MB 2.1 MB/s eta 0:00:59
   - -------------------------------------- 3.4/124.9 MB 2.4 MB/s eta 0:00:52
   - -------------------------------------- 3.9/124.9 MB 2.3 MB/s eta 0:00:52
   - -------------------------------------- 4.5/124.9 MB 2.3 MB/s eta 0:00:52
   - -------------------------------------- 5.0/124.9 MB 2.4 MB/s eta 0:00:50
   - -------------------------------------- 5.2/124.9 MB 2.4 MB/s eta 0:00:50
   - -------------------------------------- 5.5/124.9 MB 2.3 MB/s eta 0

In [61]:
%%time
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=500, learning_rate=0.05)
model.fit(X_train, Y_train)

CPU times: total: 6.27 s
Wall time: 2.16 s


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [62]:
scores(model)

{'train_rmsle': 0.01420858430218418, 'val_rmsle': 0.1259427253137462}

In [98]:
df = model.predict(test)
output3 = pd.DataFrame({"Id" : test["Id"],"SalePrice" : df })
output3.to_csv('submissionXGB.csv')

In [63]:
%%time
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    "n_estimators": [100, 500, 1000],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 10],
    "subsample": [0.7, 1],
    "colsample_bytree": [0.5, 1]
}
search = RandomizedSearchCV(XGBRegressor(), param_distributions=param_grid, n_iter=10, cv=5)
search.fit(X_train, Y_train)
best_model = search.best_estimator_

CPU times: total: 7min 54s
Wall time: 2min 51s


In [78]:
scores(search)

{'train_rmsle': 0.04589922142854131,
 'val_rmsle': 0.11381639186793845,
 'train_r2': 0.9928064942359924,
 'valid_r2': 0.9207742214202881}

In [88]:
df1 = search.predict(test)

In [89]:
df1

array([127122.586, 157303.64 , 184022.12 , ..., 167974.   , 121004.81 ,
       226517.6  ], dtype=float32)

In [90]:
output4 = pd.DataFrame({"Id" : test["Id"],"SalePrice" : df1 })
output4.to_csv('submissionXGB2.csv')

Letus try by training with all the data we have.

In [72]:
X_trainandvalid = pd.concat([X_train,X_valid],ignore_index=True)
Y_trainandvalid = pd.concat([Y_train,Y_valid],ignore_index=True)
X_trainandvalid.shape,Y_trainandvalid.shape

((1460, 80), (1460,))

In [66]:
%%time
model_full = RandomForestRegressor(n_jobs=-1,random_state=42)
model_full.fit(X_trainandvalid,Y_trainandvalid)

CPU times: total: 6.02 s
Wall time: 1.83 s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [100]:
df2 = model_full.predict(test)
output5 = pd.DataFrame({"Id" : test["Id"],"SalePrice" : df2 })
output5.to_csv('submissionRFR_full.csv')

In [67]:
%%time
from xgboost import XGBRegressor
modelXGBfull = XGBRegressor(n_estimators=500, learning_rate=0.05)
modelXGBfull.fit(X_trainandvalid, Y_trainandvalid)

CPU times: total: 6.47 s
Wall time: 2.19 s


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [104]:
df3 = modelXGBfull.predict(test)
output3 = pd.DataFrame({"Id" : test["Id"],"SalePrice" : df3 })
output3.to_csv('submissionXGB_full.csv')

In [71]:
df3 = modelXGBfull.predict(test)
print(f"df3 shape: {df3.shape}")

df3 shape: (1459,)


In [68]:
%%time
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    "n_estimators": [100, 500, 1000],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 10],
    "subsample": [0.7, 1],
    "colsample_bytree": [0.5, 1]
}
modelXGBfull2 = RandomizedSearchCV(XGBRegressor(), param_distributions=param_grid, n_iter=10, cv=5)
modelXGBfull2.fit(X_trainandvalid, Y_trainandvalid)
best_model = modelXGBfull2.best_estimator_

CPU times: total: 5min 49s
Wall time: 2min 10s


In [107]:
df4 = modelXGBfull2.predict(test)
output4 = pd.DataFrame({"Id" : test["Id"],"SalePrice" : df4 })
output4.to_csv('submissionXGB_full2.csv')

NameError: name 'Y_test' is not defined

ValueError: Found input variables with inconsistent numbers of samples: [1460, 1459]